### Purpose: Obtain the data dictionary using groq, then enrich with data from stockanalysis.com

In [14]:
# import necessary libraries
import numpy as np 
import pandas as pd
import os
import zipfile
from matplotlib import pyplot as plt

import re
from concurrent.futures import ThreadPoolExecutor

from groq import Groq  # Import Groq client



In [15]:

# Initialize the Groq client with your API key
client = Groq(api_key="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")  # Replace with your actual Groq API key

def query_using_groq(user_input,macro=False,ext='us/stocks'):
    """
    Use the Groq API to fix the SQL query or generate a new one.
    """
        
    llm_content =  "You are a private analyst at a hedge fund and you research about macroeconomic, provide the following basic information \
                                for a given macro feature as a list of elements\
                            Macro ticker: The ticker of the macro (e.g CPIYUS).    \
                            Full meaning: The definition of the macro item. " if macro else                         "You are a private analyst at a hedge fund and you research about stocks and etfs ({ext}), provide the following basic information \
                                for a given ticker as a list of elements\
                            Ticker Symbol: Unique identifier for the stock, ETF, or asset (e.g., EWZS).\
                            Exchange: The stock exchange where the ticker is listed (e.g., NYSE, NASDAQ).\
                            Asset Class: Type of financial instrument (e.g., Stock, ETF, Bond, Commodity, Crypto).\
                            Currency: The currency in which the asset is traded (e.g., USD, EUR).\
                            Country: The country of origin for the asset (e.g., USA, Brazil).\
                            Sector: Economic sector (e.g., Technology, Healthcare, Financials).\
                            Industry: More specific industry classification (e.g., Software, Pharmaceuticals).\
                            Full Name: Full company or fund name (e.g., iShares MSCI Brazil Small-Cap ETF).\
                                "
    try:
        # Create a completion request with the user prompt or SQL code
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile", #"gemma2-9b-it" "llama3-70b-8192"
            messages=[
                {
                    "role": "system",
                    "content": (llm_content)
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=False,
            stop=None,
        )

        # Extract response from Groq
        fixed_query = completion.choices[0].message.content
        return fixed_query
    except Exception as e:
        return user_input, f"Error: {str(e)}"


In [16]:

# Function to extract information from LLM output
def extract_ticker_info(llm_output, macro=False):
    equity_patterns = {
        "Ticker Symbol": r"Ticker Symbol:\s*(\S+)",
        "Exchange": r"Exchange:\s*(.+)",
        "Asset Class": r"Asset Class:\s*(.+)",
        "Currency": r"Currency:\s*(\S+)",
        "Country": r"Country:\s*(.+)",
        "Sector": r"Sector:\s*(.+)",
        "Industry": r"Industry:\s*(.+)",
        "Full Name": r"Full Name:\s*(.+)",
    }

    macro_patterns = {
        "Macro Ticker": r"Macro Ticker:\s*(\S+)",
        "Definition": r"Full Meaning:\s*(.+)",
    }
    
    patterns = macro_patterns if macro else equity_patterns
    
    extracted_info = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, llm_output)
        if match:
            extracted_info[key] = match.group(1).strip()
    
    return extracted_info


In [17]:

# Function to process a single ticker
def process_ticker(ticker, macro=False, ext='us/stocks'):
    try:
        llm_output = query_using_groq(user_input=ticker, macro=macro,ext=ext)
        llm_output = llm_output.replace('*','')
        info = extract_ticker_info(llm_output,macro)

        if macro:
                return {
            "Macro Ticker": info.get("Macro Ticker", ""),
            "Definition": info.get("Definition", ""),
            "Status": True,
                    }
        else:
            return {
                "Ticker Symbol": info.get("Ticker Symbol", ""),
                "Exchange": info.get("Exchange", ""),
                "Asset Class": info.get("Asset Class", ""),
                "Currency": info.get("Currency", ""),
                "Country": info.get("Country", ""),
                "Sector": info.get("Sector", ""),
                "Industry": info.get("Industry", ""),
                "Full Name": info.get("Full Name", ""),
                "Status": True,
            }
    except Exception as e:
        return {"Status": False, "Error": str(e)}

# Parallel processing for all tickers
def process_tickers_in_parallel(files,macro,ext):
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_ticker, ticker, macro,ext) for ticker in files]
        for future in futures:
            results.append(future.result())
    return results

In [22]:

def obtain_data_dictionary(folder_path = 'us/etf/', macro=False):
    # folder_path = 'us/etf/'

    # Check if the folder exists
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        # List all files in the folder
        files = os.listdir(folder_path)
        
        # Filter out directories
        limit = 4 if macro else 8
        files = [f[:-limit] for f in files if os.path.isfile(os.path.join(folder_path, f))]

    print(files)

    if (macro):

        results = process_tickers_in_parallel(files, macro=True, ext=folder_path)

        # Collate results into separate lists
        macro_ticker = []
        definition = []

        for result in results:
            if result["Status"]:
                macro_ticker.append(result["Macro Ticker"])
                definition.append(result["Definition"])

        dictionary_df = pd.DataFrame({
            'macro_ticker':macro_ticker,
            'definition': definition
        })

    else:  

        # Example usage
        # files = ["AAPL", "MSFT", "GOOGL"]  # Replace with your list of tickers
        results = process_tickers_in_parallel(files, macro=False, ext=folder_path)

        # Collate results into separate lists
        symbol = []
        exchange = []
        asset_class = []
        currency = []
        country = []
        sector = []
        industry = []
        fullname = []

        for result in results:
            if result["Status"]:
                symbol.append(result["Ticker Symbol"])
                exchange.append(result["Exchange"])
                asset_class.append(result["Asset Class"])
                currency.append(result["Currency"])
                country.append(result["Country"])
                sector.append(result["Sector"])
                industry.append(result["Industry"])
                fullname.append(result["Full Name"])

        dictionary_df = pd.DataFrame({
            'symbol':symbol,
            'exchange': exchange,
            'asset_class': asset_class,
            'currency': currency,
            'country':country,
            'sector': sector,
            'industry': industry,
            'fullname': fullname
        })

    return dictionary_df

In [54]:
df = obtain_data_dictionary(folder_path='macro/us', macro=True)

['ADPEUS', 'AVHEUS', 'AVWHUS', 'BSIMUS', 'BSIYUS', 'CLINUS', 'CNCIUS', 'CNCRUS', 'CPCMUS', 'CPCYUS', 'CPIMUS', 'CPIYUS', 'CPUMUS', 'CRABUS', 'CTCLUS', 'CTSPUS', 'DGLMUS', 'DGOMUS', 'EMCIUS', 'ESMIUS', 'EXPMUS', 'EXPRUS', 'EXPYUS', 'FDPHUS', 'FDRHUS', 'FGOMUS', 'GDDQUS', 'GDPQUS', 'GDPYUS', 'GVBGUS', 'HBPMUS', 'HOESUS', 'HONSUS', 'HOPMUS', 'HOPQUS', 'HOSMUS', 'IMPRUS', 'INJCUS', 'INPMUS', 'INPYUS', 'INRTUS', 'IPIMUS', 'IPIYUS', 'ISMNUS', 'ISNFUS', 'LDIIUS', 'MFPMUS', 'NAHBUS', 'NFPMUS', 'NGINUS', 'PCDMUS', 'PCDYUS', 'PCEQUS', 'PCEYUS', 'PECYUS', 'PMCHUS', 'PMCPUS', 'PMMNUS', 'PMSRUS', 'PPIMUS', 'PPIYUS', 'PSIMUS', 'PSSMUS', 'RSAMUS', 'RSAYUS', 'RSLMUS', 'RSLYUS', 'S20YUS', 'TNLFUS', 'TNTFUS', 'TRBNUS', 'ULCQUS', 'ULCYUS', 'UMCCUS', 'UNRTUS', 'WHIMUS', 'WHIYUS', 'WHSMUS', 'WHSYUS']


In [57]:
df.to_csv('data_dictionary/us_macro.csv')

In [19]:
uk_macro_df = obtain_data_dictionary(folder_path='macro/uk', macro=True)

['BSNQUK', 'BSNYUK', 'CBDTUK', 'CBIIUK', 'CPABUK', 'CPCYUK', 'CPIMUK', 'CPIYUK', 'CRABUK', 'EXPRUK', 'GDPQUK', 'GDPYUK', 'IMPRUK', 'INPMUK', 'INPYUK', 'INRTUK', 'M4SMUK', 'M4SYUK', 'MGAPUK', 'MNPMUK', 'MNPYUK', 'NTPMUK', 'NTPYUK', 'PMMNUK', 'PMSRUK', 'PPIMUK', 'PPIYUK', 'PPNMUK', 'PPNYUK', 'RSAMUK', 'RSAYUK', 'RSLMUK', 'RSLYUK', 'TRBNUK', 'UNRTUK']


In [59]:
uk_macro_df.to_csv('data_dictionary/uk_macro.csv')

In [20]:
uk_macro_df

,macro_ticker,definition
0,,
1,,
2,CBDTUK,"Corporate Bonds Total Yield UK, which refers t..."
3,CBIIUK,UK Consumer Price Index Including Housing Cost...
4,CPABUK,Consumer Price Index Above Basket (United King...
5,CPCYUK,"Consumer Price Index, Core, for the United Kin..."
6,CPIMUK,Consumer Price Index for the United Kingdom
7,CPIYUK,"Consumer Price Index for the United Kingdom, w..."
8,CRABUK,Composite Rate of Bank Accounts in the United ...
9,,


In [23]:
uk_stocks_df = obtain_data_dictionary(folder_path='uk/stocks', macro=False)

['0A1U', '0A2O', '0A2S', '0A2Z', '0A6L', '0G2Z', '0HCI', '0HD6', '0HL1', '0HR3', '0I5O', '0IPT', '0IX0', '0J77', '0JKF', '0JOQ', '0KIZ', '0M8V', '0MV6', '0N9V', '0NOF', '0O86', '0P4F', '0P5L', '0Q16', '0Q1F', '0Q1N', '0Q1S', '0QAH', '0QRA', '0QYF', '0QYI', '0QYP', '0QZ0', '0QZ1', '0QZ6', '0QZI', '0QZK', '0QZO', '0R0E', '0R0K', '0R0T', '0R0X', '0R16', '0R1M', '0R1O', '0R1Z', '0R24', '0R2V', '0R2Z', '0R34', '0R37', '0R3D', '0R3G', '0R8N', '0R9T', '0R9U', '0RGI', '0RHU', '0RIH', '0RNH', '0RPZ', '0RYA', '1SN', '3IN', '4BB', '4GBL', '53GW', '80M', '87IP', '88E', 'AA4', 'AADV', 'AAEV', 'AAF', 'AAIF', 'AAL', 'AAS', 'AATG', 'AAU', 'AAVC', 'AAZ', 'ABDN', 'ABDP', 'ABDX', 'ABF', 'ACG', 'ACP', 'ACSO', 'ADA', 'ADF', 'ADIG', 'ADM', 'ADT1', 'ADVT', 'AEET', 'AEI', 'AEO', 'AEP', 'AERI', 'AERS', 'AET', 'AEWU', 'AEX', 'AFC', 'AFM', 'AFN', 'AFP', 'AFRK', 'AFRN', 'AGFX', 'AGL', 'AGR', 'AGT', 'AGTA', 'AGVI', 'AGY', 'AGZI', 'AHT', 'AIBG', 'AIE', 'AIEA', 'AIQ', 'AIRE', 'AJB', 'AJOT', 'ALBA', 'ALF', 'ALFA', 'A

In [24]:
uk_stocks_df

,symbol,exchange,asset_class,currency,country,sector,industry,fullname
0,0A1U,Not available (assuming it's a non-US exchange...,Stock or ETF ( unable to determine without add...,GBP,United Kingdom,Not available (requires more specific informat...,Not available (requires more specific informat...,Not available ( unable to determine without ad...
1,0A2O,Not Found ( POSSIBLY LSE - London Stock Exchange),Stock,GBP,United Kingdom,Not Found,Not Found,Not Found
2,0A2S,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Financials,Investment Management,iShares MSCI Australia 0-5 Year Corporate Bond...
3,0A2Z,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Financials,Investment Trusts,Allianz Technology Trust plc
4,0A6L,LSE (London Stock Exchange),ETF,GBP,United Kingdom,Financials,Investment Trusts,iShares MSCI China A ETF
5,0G2Z,LSE (London Stock Exchange),ETF,GBP,United Kingdom,Financials,Investment Trusts,iShares MSCI Brazil ETF
6,0HCI,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Equity,ETF - Emerging Markets,iShares MSCI AC Far East ex Japan ETF
7,0HD6,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Commodities,Broad Commodities,iShares CorePhysical Gold ETC
8,0HL1,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Equity,Emerging Markets,iShares MSCI Brazil ETF
9,0HR3,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Financials,Investment Trusts,iShares Core S&P 500 UCITS ETF ( DISTRIBUTING ...


In [10]:
uk_stocks_df

,symbol,exchange,asset_class,currency,country,sector,industry,fullname
0,0A1U,London Stock Exchange (LSE),ETF,GBP,United Kingdom,Financials,Exchange Traded Funds,iShares MSCI Australia 0-5 Year BB-Bond UCITS ...
1,0A2O,Singapore Exchange (SGX),Stock,Singapore,Singapore,Industrials,Marine Transportation,Ntingham Fortune SGX:0A2O is not widely known ...
2,0A2S,Unknown (please provide more context or inform...,Unknown (please provide more context or inform...,Unknown,Unknown (please provide more context or inform...,Unknown (please provide more context or inform...,Unknown (please provide more context or inform...,Unknown (please provide more context or inform...
3,0A2Z,"Not available (possibly a non-US exchange, e.g...",Stock,GBP,United Kingdom,Not available,Not available,Not available
4,0A6L,LSE (London Stock Exchange),ETF,GBP,United Kingdom,Commodities,Broad Commodities,iShares Ashmore Emerging Markets Local Currenc...
...,...,...,...,...,...,...,...,...
111,JSGI,NASDAQ,ETF,USD,USA,Industrials,Industrials,Invesco Global Strategic Income Achievers ETF
112,JZCP,None (JZCP is not a currently trading security...,Stock,USD,United States,Not available,Not available,Jazz Pharmaceuticals plc is not associated wit...
113,KGH,NYSE,Stock,USD,Norway,Energy,Oil and Gas Equipment and Services,"Kinghorn Group Holdings Limited, however it se..."
114,KLR,NYSE,ETF,USD,USA,Energy,Oil and Gas Exploration and Production,Kpler Ltd


In [7]:
uk_etfs_df = obtain_data_dictionary(folder_path='uk/etf', macro=False)

['100D', '100H', '1AMZ', '1ARG', '1ARK', '1ARW', '1BRN', '1COI', '1GIS', '1GOO', '1MCS', '1MSF', '1NGL', '1NIO', '1OIS', '1PAS', '1TSL', '2AAP', '2AMD', '2AME', '2AMZ', '2BAB', '2BAE', '2BRE', '2BRK', '2BRT', '2BUL', '2CAR', '2FB', '2GOO', '2GS', '2JPM', '2LZM', '2MCL', '2MSF', '2MU', '2MUE', '2NFL', '2NGA', '2NVD', '2OIE', '2OIG', '2OIL', '2PAL', '2STE', '2STR', '2STS', '2SZM', '2TRV', '2TSE', '2TSL', '2UKL', '2UKS', '2VIS', '30GB', '32GH', '3AAP', '3ABE', '3ABN', '3ADE', '3AMD', '3AME', '3AMZ', '3APE', '3ARE', '3ARK', '3ARM', '3BA', '3BAB', '3BAE', '3BAL', '3BAS', '3BBE', '3BID', '3BLR', '3BP', '3BPE', '3BRL', '3BRS', '3BSR', '3CFL', '3CHE', '3CHI', '3CNE', '3CON', '3CRE', '3CRM', '3DAX', '3DEG', '3DEL', '3DES', '3DGG', '3DGL', '3DIE', '3DIS', '3DUG', '3DUS', '3DXE', '3EML', '3EMS', '3EUL', '3EUS', '3FB', '3FBE', '3FNE', '3FNG', '3FNP', '3FTE', '3FTG', '3FTP', '3GDE', '3GDX', '3GFM', '3GIL', '3GIS', '3GLD', '3GLE', '3GME', '3GMP', '3GOE', '3GOL', '3GOO', '3GOS', '3HCL', '3HCS', '3IND

In [8]:
uk_etfs_df = obtain_data_dictionary(folder_path='uk/etf', macro=False)


['100D', '100H', '1AMZ', '1ARG', '1ARK', '1ARW', '1BRN', '1COI', '1GIS', '1GOO', '1MCS', '1MSF', '1NGL', '1NIO', '1OIS', '1PAS', '1TSL', '2AAP', '2AMD', '2AME', '2AMZ', '2BAB', '2BAE', '2BRE', '2BRK', '2BRT', '2BUL', '2CAR', '2FB', '2GOO', '2GS', '2JPM', '2LZM', '2MCL', '2MSF', '2MU', '2MUE', '2NFL', '2NGA', '2NVD', '2OIE', '2OIG', '2OIL', '2PAL', '2STE', '2STR', '2STS', '2SZM', '2TRV', '2TSE', '2TSL', '2UKL', '2UKS', '2VIS', '30GB', '32GH', '3AAP', '3ABE', '3ABN', '3ADE', '3AMD', '3AME', '3AMZ', '3APE', '3ARE', '3ARK', '3ARM', '3BA', '3BAB', '3BAE', '3BAL', '3BAS', '3BBE', '3BID', '3BLR', '3BP', '3BPE', '3BRL', '3BRS', '3BSR', '3CFL', '3CHE', '3CHI', '3CNE', '3CON', '3CRE', '3CRM', '3DAX', '3DEG', '3DEL', '3DES', '3DGG', '3DGL', '3DIE', '3DIS', '3DUG', '3DUS', '3DXE', '3EML', '3EMS', '3EUL', '3EUS', '3FB', '3FBE', '3FNE', '3FNG', '3FNP', '3FTE', '3FTG', '3FTP', '3GDE', '3GDX', '3GFM', '3GIL', '3GIS', '3GLD', '3GLE', '3GME', '3GMP', '3GOE', '3GOL', '3GOO', '3GOS', '3HCL', '3HCS', '3IND

In [9]:
uk_etfs_df

,symbol,exchange,asset_class,currency,country,sector,industry,fullname


In [45]:
# llm_output = query_using_groq(user_input='ADPEUS',macro=True)
# llm_output = query_using_groq(user_input='GDPYUS',macro=True)
llm_output = query_using_groq(user_input='AVWHUS',macro=True)

In [46]:
llm_output

'Here is the information for the macro feature AVWHUS:\n\n* **Macro Ticker:** AVWHUS\n* **Full Meaning:** Average Weekly Hours of Production and Nonsupervisory Employees: Manufacturing (USA)'